In [1]:
# Import the wrangled dataset from Monday

DATA_PATH = './notebooks/vgsalesWrangleMonday.csv'
import pandas as pd
import numpy as np
df = pd.read_csv(DATA_PATH, na_values=['Unknown'])
pd.set_option('display.max_columns', 23)
pd.set_option("display.max_rows", 100)
df.head()

,Rank,Name,Genre,ESRB_Rating,Platform,Publisher,Developer,Critic_Score,User_Score,Total_Shipped,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Vgchartzscore,All_Sales,Sold_100k,All_Sales_Transformed,Average_Score,Number_Platforms,Number_Games_From_Publisher
0,1,Wii Sports,Sports,E,Wii,Nintendo,Nintendo EAD,7.7,NaN,82.86,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,82.86,True,4.429149,7.700000,1,753.0
1,2,Super Mario Bros.,Platform,NaN,NES,Nintendo,Nintendo EAD,10.0,NaN,40.24,NaN,NaN,NaN,NaN,NaN,1985.0,NaN,40.24,True,3.719409,10.000000,1,753.0
2,3,Mario Kart Wii,Racing,E,Wii,Nintendo,Nintendo EAD,8.2,9.1,37.14,NaN,NaN,NaN,NaN,NaN,2008.0,8.7,37.14,True,3.641264,8.666667,1,753.0
3,4,PlayerUnknown's Battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,36.60,NaN,NaN,NaN,NaN,NaN,2017.0,NaN,36.60,True,3.627004,NaN,3,1.0
4,5,Wii Sports Resort,Sports,E,Wii,Nintendo,Nintendo EAD,8.0,8.8,33.09,NaN,NaN,NaN,NaN,NaN,2009.0,8.8,33.09,True,3.529004,8.533333,1,753.0


In [2]:
df['Average_Score'] = df['Average_Score'].fillna(-0.1)

In [3]:
df.head()

,Rank,Name,Genre,ESRB_Rating,Platform,Publisher,Developer,Critic_Score,User_Score,Total_Shipped,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Vgchartzscore,All_Sales,Sold_100k,All_Sales_Transformed,Average_Score,Number_Platforms,Number_Games_From_Publisher
0,1,Wii Sports,Sports,E,Wii,Nintendo,Nintendo EAD,7.7,NaN,82.86,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,82.86,True,4.429149,7.700000,1,753.0
1,2,Super Mario Bros.,Platform,NaN,NES,Nintendo,Nintendo EAD,10.0,NaN,40.24,NaN,NaN,NaN,NaN,NaN,1985.0,NaN,40.24,True,3.719409,10.000000,1,753.0
2,3,Mario Kart Wii,Racing,E,Wii,Nintendo,Nintendo EAD,8.2,9.1,37.14,NaN,NaN,NaN,NaN,NaN,2008.0,8.7,37.14,True,3.641264,8.666667,1,753.0
3,4,PlayerUnknown's Battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,36.60,NaN,NaN,NaN,NaN,NaN,2017.0,NaN,36.60,True,3.627004,-0.100000,3,1.0
4,5,Wii Sports Resort,Sports,E,Wii,Nintendo,Nintendo EAD,8.0,8.8,33.09,NaN,NaN,NaN,NaN,NaN,2009.0,8.8,33.09,True,3.529004,8.533333,1,753.0


In [5]:
# Define features/target

target='Sold_100k'

features=['Genre','ESRB_Rating','Platform','Year','Average_Score','Number_Platforms','Number_Games_From_Publisher']

In [6]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df,random_state=84)
train.shape, val.shape

((14896, 23), (4966, 23))

In [7]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]

In [8]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

pipelineTrainVal = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=200,random_state=235,n_jobs=-1)
)

# Fit on train
pipelineTrainVal.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Genre', 'ESRB_Rating', 'Platform'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'Genre',
                                          'data_type': dtype('O'),
                                          'mapping': Action               1
Role-Playing         2
Fighting             3
Simulation           4
Adventure            5
Sports               6
Misc                 7
Platform             8
Strategy             9
Action-Adventure    10
Puzzle              11
Shooter             12
Visual Nove...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=

In [9]:
# Score on validate
print('Validation Accuracy', pipelineTrainVal.score(X_val, y_val))

# Make a prediction
y_pred = pipelineTrainVal.predict(X_val)

Validation Accuracy 0.7784937575513492


In [10]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

X = df[features]
y = df[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=200,random_state=235,n_jobs=-1)
)

# Fit on train
pipeline.fit(X, y)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Genre', 'ESRB_Rating', 'Platform'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'Genre',
                                          'data_type': dtype('O'),
                                          'mapping': Sports               1
Platform             2
Racing               3
Shooter              4
Role-Playing         5
Puzzle               6
Misc                 7
Party                8
Simulation           9
Action              10
Action-Adventure    11
Fighting            12
Strategy            13
Advent...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                     

In [11]:
from joblib import dump
dump(pipeline, 'pipeline.joblib', compress=True)

['pipeline.joblib']

In [8]:
y_pred

array([ True, False, False, ..., False, False,  True])

In [49]:
X_val.head(20)

,Genre,ESRB_Rating,Platform,Year,Average_Score,Number_Platforms,Number_Games_From_Publisher
247,Sports,E,PS3,2010.0,8.7,7,534.0
19439,Adventure,NaN,PSV,2017.0,NaN,1,47.0
12326,Racing,T,PS2,2003.0,NaN,3,243.0
17008,Racing,E,PS4,2017.0,NaN,2,59.0
3774,Sports,E,XB,2002.0,NaN,1,104.0
9390,Shooter,M,GC,2005.0,7.9,5,1027.0
5431,Misc,E,GBA,2001.0,NaN,1,767.0
15576,Sports,E,GBA,2004.0,NaN,1,2.0
3245,Sports,E,Wii,2010.0,6.8,5,534.0
15266,Racing,E,PS2,2001.0,NaN,1,24.0


In [31]:
transformers = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean')
)

X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.transform(X_val)

model = RandomForestClassifier(n_estimators=100,random_state=235,n_jobs=-1)
model.fit(X_train_transformed,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=235, verbose=0,
                       warm_start=False)

In [34]:
import eli5
from eli5.sklearn import PermutationImportance
permuter = PermutationImportance(
    model,
    scoring='accuracy',
    n_iter=5,
    random_state=84
)

permuter.fit(X_val_transformed,y_val)

feature_names = X_val.columns.tolist()
pd.Series(permuter.feature_importances_, feature_names).sort_values(ascending=False)

Number_Games_From_Publisher    0.078453
Year                           0.073258
Platform                       0.072010
ESRB_Rating                    0.038099
Average_Score                  0.031051
Number_Platforms               0.029601
Genre                          0.015103
dtype: float64

In [7]:
from joblib import dump
dump(pipeline, 'pipeline.joblib', compress=True)

['pipeline.joblib']

In [8]:
import joblib
import sklearn
import category_encoders as ce
print(f'joblib=={joblib.__version__}')
print(f'scikit-learn=={sklearn.__version__}')
print(f'category_encoders=={ce.__version__}')

joblib==0.13.2
scikit-learn==0.21.3
category_encoders==2.1.0


In [9]:
df['Genre'].value_counts()

Action              2979
Sports              2558
Misc                2224
Adventure           1848
Role-Playing        1808
Shooter             1550
Racing              1423
Simulation          1134
Platform            1123
Fighting             945
Strategy             866
Puzzle               723
Action-Adventure     334
Music                144
Visual Novel         110
MMO                   44
Party                 38
Sandbox                6
Education              3
Board Game             2
Name: Genre, dtype: int64

In [11]:
df['ESRB_Rating'].value_counts()

E      5927
T      3934
M      2117
E10    1884
RP       38
EC       25
Name: ESRB_Rating, dtype: int64

In [12]:
df['Platform'].value_counts()

DS      2265
PS2     2218
PC      2142
PS3     1373
Wii     1360
PSP     1318
X360    1310
PS      1207
PS4      940
GBA      858
XB       836
PSV      645
3DS      627
GC       563
XOne     539
N64      322
NS       281
SNES     241
SAT      175
WiiU     161
2600     133
NES      102
GB        94
DC        52
GEN       31
NG        12
PSN       10
GBC        9
XBL        7
WS         7
SCD        6
Mob        4
3DO        4
VC         3
PCE        2
Amig       1
OSX        1
WW         1
GG         1
PCFX       1
Name: Platform, dtype: int64

In [14]:
y_val.value_counts(normalize=True)

True     0.648409
False    0.351591
Name: Sold_100k, dtype: float64

In [36]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

X = df[features]
y = df[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=200,random_state=235,n_jobs=-1)
)

# Fit on train
pipeline.fit(X, y)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Genre', 'ESRB_Rating', 'Platform'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'Genre',
                                          'data_type': dtype('O'),
                                          'mapping': Sports               1
Platform             2
Racing               3
Shooter              4
Role-Playing         5
Puzzle               6
Misc                 7
Party                8
Simulation           9
Action              10
Action-Adventure    11
Fighting            12
Strategy            13
Advent...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                     

In [37]:
from joblib import dump
dump(pipeline, 'pipeline.joblib', compress=True)

['pipeline.joblib']

In [38]:
df['Sold_100k'].value_counts()

True     12642
False     7220
Name: Sold_100k, dtype: int64

In [41]:
df[df['Sold_100k']==False].describe()

,Rank,Critic_Score,User_Score,Total_Shipped,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Vgchartzscore,All_Sales,All_Sales_Transformed,Average_Score,Number_Platforms,Number_Games_From_Publisher
count,7220.000000,771.000000,31.000000,7.000000,7213.000000,3497.000000,3348.000000,2938.000000,4657.000000,7217.000000,63.000000,7220.000000,7220.000000,817.000000,7220.000000,7093.000000
mean,16252.500000,6.608820,7.661290,0.067143,0.040603,0.035216,0.016237,0.033131,0.002899,2009.035195,6.366667,0.040629,0.039531,6.601816,1.726039,252.052164
std,2084.378804,1.591373,1.818915,0.026277,0.025297,0.020709,0.014915,0.023206,0.004804,5.703199,1.534759,0.025310,0.024196,1.604299,1.190191,306.743706
min,12643.000000,1.000000,2.000000,0.030000,0.010000,0.000000,0.000000,0.000000,0.000000,1970.000000,3.000000,0.010000,0.009950,1.000000,1.000000,1.000000
25%,14447.750000,5.800000,7.050000,0.050000,0.020000,0.020000,0.010000,0.010000,0.000000,2006.000000,5.500000,0.020000,0.019803,5.800000,1.000000,26.000000
50%,16252.500000,6.900000,8.000000,0.080000,0.040000,0.030000,0.010000,0.030000,0.000000,2009.000000,6.800000,0.040000,0.039221,6.900000,1.000000,110.000000
75%,18057.250000,7.800000,8.900000,0.085000,0.060000,0.050000,0.020000,0.050000,0.010000,2013.000000,7.400000,0.060000,0.058269,7.800000,2.000000,373.000000
max,19862.000000,9.500000,10.000000,0.090000,0.090000,0.090000,0.090000,0.090000,0.060000,2020.000000,9.500000,0.090000,0.086178,10.000000,9.000000,1027.000000


In [42]:
from joblib import dump
dump(pipeline, 'pipeline.joblib', compress=True)

,Name,Genre,ESRB_Rating,Platform,Publisher,Developer,Sold_100k
count,7220,7220,4017,7220,7093,6698,7220
unique,6163,19,6,37,591,1888,1
top,Rugby World Cup 2015,Adventure,E,DS,Konami,Konami,False
freq,6,1022,1808,1092,301,168,7220


In [ ]:
df[df['Sold_100k']==False].describe(exclude='number')